# ROC curves, AUC (area under the curve) and MCC (Matthews' correlation coefficient)

In this notebook, we illustrate how to:

1. draw the ROC curve
2. calculate the AUC (area under the curve)
3. claculate the MCC (Matthew's correlation coefficient)

## Loading libraries

First of all, we load some necessary general libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Get the data

We are using results from a mock binary classification problem.
This was based on the [breast cancer Wisconsin dataset](https://github.com/scikit-learn/scikit-learn/blob/6e9039160f0dfc3153643143af4cfdca941d2045/sklearn/datasets/data/breast_cancer.csv) from the Python library `sklearn`.

In this dataset, the objective is to diagnose the status of breast cancer:

- `0`: malignant cancer
- `1`: benign cancer

The dataset contains **569 examples**:

- 212 malignant
- 357 benign

And the prediction (classification) is based on 30 numeric features related to the cancer lesions (size, shape etc.: full description can be found [here](https://scikit-learn.org/1.5/datasets/toy_dataset.html#breast-cancer-dataset)).

Actually, in this example we used a random subset of the 30 features, by selecting 8 features: in this way, the problem was harder, and we obtained more classification errors, which is instrumental to the illustration of different metrics to measure model performance.

The dataset is imbalanced: the ratio between the two classes is not 1, but 0.6.
Again, this is instrumental in showing the relative advantage of using different performance metrics rather than just looking at the error rate / overall accuracy.

In [ ]:
DATASET_URL = 'https://raw.githubusercontent.com/ne1s0n/bioinformateachers/refs/heads/main/dlb/data/predictions.csv'

The dataframe contains:

- the **original test observation** (the "truth": malignant or benign)
- the **predicted class** (binary)
- the **two probabilities**: of being '0' (malignant) or '1' (benign)

In [ ]:
bc_data = pd.read_csv(DATASET_URL)
bc_data.head()

The test dataset was generated by taking a 20% random subset of the data: **114 test examples**.

In [ ]:
len(bc_data)

---

Let's get a look at the original **confusion matrix**: first we get the two vectors of predictions and observations, and then construct the matrix of correct predictions (diagonal) and errors (off diagonal).

In [ ]:
y_test = np.array(bc_data['y_test'])
y_pred = np.array(bc_data['y_pred'])

In [ ]:
# import the metrics class
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

---

The next bit will be to bring the **probabilities of prediction** in the game, not just the predicted classes.

Keep in mind that the predicted classes have been obtained based on the **0.5 threshold**: each test example is attributed to the class whose probability is larger than 50%.

In [ ]:
y_probs = np.array(bc_data[bc_data.columns[2:4]])
print(y_probs[0:5,:])

---

### ROC curves

This is a binary classification problem, therefore the model usually focuses on the probability for just one class (being the other unambiguously obtained as reciprocal to 1).
Most commonly, the probability of class `1` ("case") is modeled:

$$
P(y=1 | X)
$$

Therefore, this probability is used in the calculation of the **ROC curve**.

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
prob_y_eq_1 = y_probs[:,1]

The **ROC curve** is based on looking at classification results from the perspective of all (many) classification thresholds, not just the standard 50%.

This means that probabilities ($P(y=1)$) are evaluated against threshold 0%, 0.5% 10% $\ldots$ 90%, 95%, 100%): for each threshold the **false positive rate** (FP/(FP+TN)) and the **true positive rate** (TP/(TP+FN) = 1-FNR) are calculated and then plotted against each other.

The function `roc_curve` from `sklearn` takes in input the correct test labels (the "truth") and the prediction probabilities obtained from the classification model.

This function returns the ingredients needed to draw the ROC curve: the FPR and TPR calculated against several classification thresholds (by default, 20 thresholds are considered):

In [ ]:
fpr, tpr, thrs = roc_curve(y_test, prob_y_eq_1)

We obtain **20 values** for **FPR** and for **TPR**:

In [ ]:
len(fpr)

In [ ]:
df = pd.DataFrame(np.vstack((fpr,tpr)).T, columns=['FPR','TPR'])
df.head(10)

We now have all the elements to plot the ROC curve for this classification problem.

Typically, a ROC curve is visually contrasted against chance accuracy, which for a binary classification problem is 50%, and it is represented by a straight line bisecting the plot:

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    #label="ROC curve (area = %0.2f)" % roc_auc,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
#plt.legend(loc="lower right")
plt.show()

### AUC

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
print("AUC is", roc_auc)

In [ ]:
plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()

### MCC

[TODO]